In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
t1 = torch.zeros(4,2)
print(t1)
print(t1.shape)
print(t1.dtype)
print(t1.device)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t1_gpu = t1.cuda(device)
print(t1_gpu.device)
print(t1.device)

In [ ]:
""" squeeze & unsqueeze
squeeze함수는 차원이 1인 차원을 제거해준다. 
따로 차원을 설정하지 않으면 1인 차원을 모두 제거한다. 
그리고 차원을 설정해주면 그 차원만 제거한다.

unsqueeze함수는 squeeze함수의 반대로 1인 차원을 생성하는 함수이다. 
그래서 어느 차원에 1인 차원을 생성할 지 꼭 지정해주어야한다.

"""
t2 = torch.rand(2)
print("t2: ")
print(t2)
print(t2.shape)

print("\n---after unsqueeze---")
t2_add_rank = t2.unsqueeze(0)
print(t2_add_rank)
print(t2_add_rank.shape) # 1,2

print(t2_add_rank.unsqueeze(1).shape) # 1, 1 ,2


print(t2_add_rank.squeeze(1).shape) # 1, 2
print(t2_add_rank.squeeze().shape) # 2

In [ ]:
t3 = torch.rand(1,2)
print(t3)
print(t3.shape)

print("\n---after squeeze---")
t3_delete_rank = t3.squeeze(0)
print(t3_delete_rank)
print(t3_delete_rank.shape)


In [ ]:
"""
reshape & permute
"""

t4 = torch.arange(192) # 3 * 8 * 8
print(t4,"\n", t4.shape)

t4 = t4.reshape((3,8,8))
print(f"\n---after reshape---")
print(t4, "\n" ,t4.shape)

t4 = t4.permute(1,2,0)
print("\n---after permute---")
print(t4, "\n", t4.shape)

t4 = t4.permute(2,0,1)
print(t4, "\n" ,t4.shape)